# Scraping: récupération de la page d'accueil du journal Le Soir

Dans ce notebook, nous créons un robot qui va ouvrir la page d'accueil du site du journal [Le Soir](https://www.lesoir.be/) et récupérer le titre de tous les articles du jour et les stocker dans un fichier csv.

## Imports

In [1]:
import os
#import re
#import time
import requests
from bs4 import BeautifulSoup
#import pandas as pd
from pathlib import Path

## Récupération de tous les articles de la page d'accueil



In [ ]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}


# liste où on collectera les URLs / noms de fichiers PDF
pdf_files = []

# base_url (équivalent de root_url)
base_url = "https://max.de.wilde.web.ulb.be/camille/"

response = requests.get(base_url, headers=headers)

# on parse la page
soup = BeautifulSoup(response.text, 'html.parser')

for link in soup.find_all("a", {"class":"stretched-link outline-hidden"}):
    title = link.text.strip()
    url = root_url + link.get('href')
    articles.append([url, title])

In [3]:
# Affichage du nombre d'articles récupérés
len(articles)

80

In [4]:
# Affichage des 10 premières entrées
articles[:10]

[['https://www.rtbf.be/article/l-institut-federal-des-droits-humains-met-en-garde-contre-l-erosion-insidieuse-et-preoccupante-de-l-etat-de-droit-en-belgique-11616510',
  'L\'Institut fédéral des droits humains met en garde contre l\'érosion "insidieuse et préoccupante" de l\'État de droit en Belgique'],
 ['https://www.rtbf.be/article/accord-pour-gaza-israel-devrait-rouvrir-mercredi-le-point-de-passage-de-rafah-11616457',
  'Accord pour Gaza\xa0: Israël devrait rouvrir mercredi le point de passage de Rafah'],
 ['https://www.rtbf.be/article/deux-exemplaires-du-faux-soir-mis-aux-encheres-il-s-agit-de-l-un-des-coups-les-plus-spectaculaires-de-la-resistance-11615818',
  'Deux exemplaires du "Faux Soir" mis aux enchères\xa0: "Il s’agit de l’un des coups les plus spectaculaires de la résistance"'],
 ['https://www.rtbf.be/article/jamy-gourmaud-decrypte-les-phenomenes-de-la-vie-quotidienne-dans-son-livre-365-nouveaux-jours-avec-jamy-11616519',
  'Jamy Gourmaud décrypte les phénomènes de la vie 

## Création d'un dataframe avec les liens et les titres des articles


In [5]:
df = pd.DataFrame(articles, columns=['link', 'title'])
df

,link,title
0,https://www.rtbf.be/article/l-institut-federal...,L'Institut fédéral des droits humains met en g...
1,https://www.rtbf.be/article/accord-pour-gaza-i...,Accord pour Gaza : Israël devrait rouvrir merc...
2,https://www.rtbf.be/article/deux-exemplaires-d...,"Deux exemplaires du ""Faux Soir"" mis aux enchèr..."
3,https://www.rtbf.be/article/jamy-gourmaud-decr...,Jamy Gourmaud décrypte les phénomènes de la vi...
4,https://www.rtbf.be/article/le-ministre-franck...,Le ministre Francken demande l'utilisation de ...
...,...,...
75,https://www.rtbf.behttps://auvio.rtbf.be/media...,Microbrasseries belges : le goût du défi
76,https://www.rtbf.behttps://auvio.rtbf.be/media...,Le chemin du bonheur
77,https://www.rtbf.behttps://auvio.rtbf.be/media...,The Voice Kids S03
78,https://www.rtbf.behttps://auvio.rtbf.be/media...,"Trains mythiques, paysages de légendes"


In [6]:
# Sauvegarde du dataframe dans un fichier csv
df.to_csv(f"../data/rtbf_{time.strftime('%Y%m%d')}.csv", index=False)

## Téléchargement d'un article et affichage du texte

In [7]:
# Récupération du premier lien du dataframe
article_url = df.iloc[0][0]

# Ouverture de la page
response = requests.get(article_url, headers=headers)
soup = BeautifulSoup(response.content, 'html.parser')
# Parsing du contenu
title = soup.find("h1").get_text(separator=' ')

paragraphs = []
for paragraph in soup.find_all("p", attrs={"class": None}):
    paragraphs.append(paragraph.get_text(separator=' ').strip())

content = "\n".join(paragraphs)
print(title)
print("==================================")
print(content)

C:\Users\Ing Armel Fopa\AppData\Local\Temp\ipykernel_5716\3125040403.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  article_url = df.iloc[0][0]


Belgique
Parmi les sujets de préoccupations de l'Institut  figurent notamment la non-exécution des décisions de justice, le manque de moyens alloués à cette dernière, le recours de plus en plus fréquent aux sanctions administratives ainsi que les restrictions aux libertés fondamentales comme le droit de manifester et la liberté d'expression.
L'État de droit est une des pierres angulaires d'une société juste et fonctionnelle, rappelle l'institution pour qui la vigilance est donc de mise car même dans les démocraties établies comme la Belgique certains principes fondamentaux " semblent avoir tendance à s'effriter ".
L'IFDH recommande dès lors au pouvoir exécutif de prendre les droits humains et l'État de droit comme critère central lors de l'élaboration et de la mise en œuvre des politiques car " certaines mesures prises, même lorsqu'elles paraissent bien intentionnées ou pragmatiques, peuvent miner la légitimité des politiques à long terme ", souligne l'instance.
L'organe lance par aill

### Nettoyage du texte à l'aide d'expressions régulières

In [8]:
# Suppression de tout ce qui se trouve entre parenthèses
clean_content = re.sub("\([^\)]+\)", " ", content)
# Suppression des espaces multiples
clean_content = re.sub("\s+", " ", clean_content)

print(content)
print("==================================")
print(clean_content)


Parmi les sujets de préoccupations de l'Institut  figurent notamment la non-exécution des décisions de justice, le manque de moyens alloués à cette dernière, le recours de plus en plus fréquent aux sanctions administratives ainsi que les restrictions aux libertés fondamentales comme le droit de manifester et la liberté d'expression.
L'État de droit est une des pierres angulaires d'une société juste et fonctionnelle, rappelle l'institution pour qui la vigilance est donc de mise car même dans les démocraties établies comme la Belgique certains principes fondamentaux " semblent avoir tendance à s'effriter ".
L'IFDH recommande dès lors au pouvoir exécutif de prendre les droits humains et l'État de droit comme critère central lors de l'élaboration et de la mise en œuvre des politiques car " certaines mesures prises, même lorsqu'elles paraissent bien intentionnées ou pragmatiques, peuvent miner la légitimité des politiques à long terme ", souligne l'instance.
L'organe lance par ailleurs un a

<>:2: SyntaxWarning: invalid escape sequence '\('
<>:4: SyntaxWarning: invalid escape sequence '\s'
<>:2: SyntaxWarning: invalid escape sequence '\('
<>:4: SyntaxWarning: invalid escape sequence '\s'
C:\Users\Ing Armel Fopa\AppData\Local\Temp\ipykernel_5716\3799268790.py:2: SyntaxWarning: invalid escape sequence '\('
  clean_content = re.sub("\([^\)]+\)", " ", content)
C:\Users\Ing Armel Fopa\AppData\Local\Temp\ipykernel_5716\3799268790.py:4: SyntaxWarning: invalid escape sequence '\s'
  clean_content = re.sub("\s+", " ", clean_content)


### Création d'un fichier avec le contenu de l'article


In [9]:
with open("../data/rtbf_example.txt", "w") as writer:
    writer.write(f"{title}\n\n{clean_content}")

## Pour en savoir plus

- Le web scraping avec Python: https://realpython.com/beautiful-soup-web-scraper-python/
- Tutoriel sur les expressions régulières: https://www.w3schools.com/python/python_regex.asp